In [1]:
# This python script extract the red team authentication which included auth type, logon type, logon orientation
# since the red team file only included the user, descomp, source comp information 
# After that, we will produce a list of line number that the red team log located in the auth.txt

In [ ]:
!pip uninstall -y torchdata dgl torch_geometric

!pip install -q torchdata==0.7.0

!pip install -q dgl -f https://data.dgl.ai/wheels/repo.html

!pip install -q torch_geometric

Found existing installation: torchdata 0.11.0
Uninstalling torchdata-0.11.0:
  Successfully uninstalled torchdata-0.11.0
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.7/4.7 MB 48.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 670.2/670.2 MB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 MB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 101.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 55.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 40.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 731.7/731.7 MB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.6/121.6 MB 7.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.5/56.5 MB 30.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.2/124.2 MB 13.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 196.0/196.0 MB 2.3 MB/s eta 0:00:00
  

In [ ]:
%matplotlib inline

import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from tqdm import tqdm
import dgl
import torch
import torch.nn as nn
import torch_geometric
# from torch_geometric import Hetero
import time
import os, datetime
from csv import reader

plt.rcParams['figure.figsize'] = (15, 5)
plt.rcParams['font.family'] = 'sans-serif'

pd.set_option('display.width', 5000) 
pd.set_option('display.max_columns', 60)

DGL backend not selected or invalid.  Assuming PyTorch for now.


Setting the default backend to "pytorch". You can change it in the ~/.dgl/config.json file or export the DGLBACKEND environment variable.  Valid options are: pytorch, mxnet, tensorflow (all lowercase)


# Extract red team authentication information from auth.txt

In [4]:
df_red = pd.read_csv('/kaggle/input/lmtrackerdata/redteam.txt', sep =',' , header=None)
time_set = set(df_red[0])
df_red

,0,1,2,3
0,150885,U620@DOM1,C17693,C1003
1,151036,U748@DOM1,C17693,C305
2,151648,U748@DOM1,C17693,C728
3,151993,U6115@DOM1,C17693,C1173
4,153792,U636@DOM1,C17693,C294
...,...,...,...,...
744,2552687,U655@DOM1,C17693,C828
745,2552687,U655@DOM1,C17693,C828
746,2552687,U655@DOM1,C17693,C828
747,2552687,U655@DOM1,C17693,C828


In [5]:
from csv import reader
# open file in read mode
count = 0

day_in_second = 60*60*24
day_filter = [2, 3, 6, 7, 8, 9, 10, 13, 14, 15, 16, 21, 22, 23, 27, 28, 29, 30] # only get data from this list of day
user_comp_dict = set()
comp_comp_dict = set()
user_user_dict = set()
df_red_full = []

with open('/kaggle/input/lmtrackerdata/auth.txt', 'r') as read_obj:
    # pass the file object to reader() to get the reader object
    csv_reader = reader(read_obj)
    # Iterate over each row in the csv using reader object
    for row in csv_reader:
#         if auth_filter(row, day_filter):
#             continue
        if np.ceil(int(row[0]) / day_in_second) > 40:
            break
        count = count + 1
        if count % 10000000 == 0:
            print(str(count/1000000) + "Mth lines")
        source_user     = row[1].split("@")[0]
        des_user        = row[2].split("@")[0]
        source_computer = row[3]
        des_computer    = row[4]
        if int(row[0]) in time_set:
            for i,row_red in df_red.iterrows():
                if row_red[1] == row[1] and row_red[2] == row[3] and row_red[3] == row[4]:
                    df_red_full.append(row)
            
df_red_full = pd.DataFrame(df_red_full)
# df


10.0Mth lines
20.0Mth lines
30.0Mth lines
40.0Mth lines
50.0Mth lines
60.0Mth lines
70.0Mth lines
80.0Mth lines
90.0Mth lines
100.0Mth lines
110.0Mth lines
120.0Mth lines
130.0Mth lines
140.0Mth lines
150.0Mth lines
160.0Mth lines
170.0Mth lines
180.0Mth lines
190.0Mth lines
200.0Mth lines
210.0Mth lines
220.0Mth lines
230.0Mth lines
240.0Mth lines
250.0Mth lines
260.0Mth lines
270.0Mth lines
280.0Mth lines
290.0Mth lines
300.0Mth lines
310.0Mth lines
320.0Mth lines
330.0Mth lines
340.0Mth lines
350.0Mth lines
360.0Mth lines
370.0Mth lines
380.0Mth lines
390.0Mth lines
400.0Mth lines
410.0Mth lines
420.0Mth lines
430.0Mth lines
440.0Mth lines
450.0Mth lines
460.0Mth lines
470.0Mth lines
480.0Mth lines
490.0Mth lines
500.0Mth lines
510.0Mth lines
520.0Mth lines
530.0Mth lines
540.0Mth lines
550.0Mth lines
560.0Mth lines
570.0Mth lines
580.0Mth lines
590.0Mth lines
600.0Mth lines
610.0Mth lines
620.0Mth lines
630.0Mth lines
640.0Mth lines
650.0Mth lines
660.0Mth lines
670.0Mth lines
680.

In [6]:
df_red_full.to_csv('red_team_auth.txt', header=None, index=None, sep=',', mode='a')

# Extract red team line number in the auth file

In [7]:
df_red = pd.read_csv('/kaggle/input/lmtrackerdata/redteam.txt', sep =',' , header=None)
time_set = set(df_red[0])

In [8]:
from csv import reader
# open file in read mode
count = 0

day_in_second = 60*60*24
day_filter = [2, 3, 6, 7, 8, 9, 10, 13, 14, 15, 16, 21, 22, 23, 27, 28, 29, 30] # only get data from this list of day
user_comp_dict = set()
comp_comp_dict = set()
user_user_dict = set()
df_red_full = []
red_team_line = []
with open('/kaggle/input/lmtrackerdata/auth.txt', 'r') as read_obj:
    # pass the file object to reader() to get the reader object
    csv_reader = reader(read_obj)
    # Iterate over each row in the csv using reader object
    for row in csv_reader:
#         if auth_filter(row, day_filter):
#             continue
        if np.ceil(int(row[0]) / day_in_second) > 40:
            break
        count = count + 1
        if count % 10000000 == 0:
            print(str(count/1000000) + "Mth lines")
        source_user     = row[1].split("@")[0]
        des_user        = row[2].split("@")[0]
        source_computer = row[3]
        des_computer    = row[4]
        if int(row[0]) in time_set:
            for i,row_red in df_red.iterrows():
                if row_red[1] == row[1] and row_red[2] == row[3] and row_red[3] == row[4]:
                    red_team_line.append(count)

10.0Mth lines
20.0Mth lines
30.0Mth lines
40.0Mth lines
50.0Mth lines
60.0Mth lines
70.0Mth lines
80.0Mth lines
90.0Mth lines
100.0Mth lines
110.0Mth lines
120.0Mth lines
130.0Mth lines
140.0Mth lines
150.0Mth lines
160.0Mth lines
170.0Mth lines
180.0Mth lines
190.0Mth lines
200.0Mth lines
210.0Mth lines
220.0Mth lines
230.0Mth lines
240.0Mth lines
250.0Mth lines
260.0Mth lines
270.0Mth lines
280.0Mth lines
290.0Mth lines
300.0Mth lines
310.0Mth lines
320.0Mth lines
330.0Mth lines
340.0Mth lines
350.0Mth lines
360.0Mth lines
370.0Mth lines
380.0Mth lines
390.0Mth lines
400.0Mth lines
410.0Mth lines
420.0Mth lines
430.0Mth lines
440.0Mth lines
450.0Mth lines
460.0Mth lines
470.0Mth lines
480.0Mth lines
490.0Mth lines
500.0Mth lines
510.0Mth lines
520.0Mth lines
530.0Mth lines
540.0Mth lines
550.0Mth lines
560.0Mth lines
570.0Mth lines
580.0Mth lines
590.0Mth lines
600.0Mth lines
610.0Mth lines
620.0Mth lines
630.0Mth lines
640.0Mth lines
650.0Mth lines
660.0Mth lines
670.0Mth lines
680.

In [9]:
red_team_line

[29627158,
 29657342,
 29776465,
 29850299,
 30188661,
 30188661,
 30425164,
 30458239,
 30468056,
 30489740,
 30662348,
 39964826,
 39964826,
 39964826,
 40009261,
 40009261,
 40009261,
 40013598,
 40013598,
 40013598,
 41753167,
 41753167,
 43593686,
 43593686,
 43593686,
 43593686,
 43593686,
 43593686,
 43674800,
 43674800,
 43674800,
 43674800,
 43674800,
 43674800,
 43699521,
 43699521,
 43699521,
 43699521,
 43699521,
 43699521,
 43756030,
 43756030,
 43756030,
 43756030,
 43756030,
 43756030,
 43805639,
 43805639,
 43805639,
 43805639,
 43805639,
 43805639,
 43832031,
 43832031,
 43832031,
 43832031,
 43832031,
 43832031,
 43933078,
 43933078,
 43933078,
 43935948,
 43935948,
 43935948,
 44014221,
 44014221,
 44014221,
 44279087,
 45054315,
 45054315,
 80527039,
 80527039,
 80527039,
 80672009,
 80672009,
 80672009,
 80751698,
 80751698,
 80751698,
 81001389,
 81001389,
 81001389,
 81001389,
 81442880,
 81693840,
 81693840,
 81693840,
 81693840,
 81696203,
 81696203,
 81696203,

In [10]:
import pickle

with open("red_team_list.pkl", "wb") as fp:   #Pickling
    pickle.dump(red_team_line, fp)